## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=['EIN','NAME'], inplace=True)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ['T13','T12','T2','T25','T14','T29','T15','T17','T9']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [7]:
# I find it helpful just to define the list and set the cutoff.
classification_list = list(application_df['CLASSIFICATION'].value_counts().index)
classification_counts = list(application_df['CLASSIFICATION'].value_counts().values)
cutoff_value = 250

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = []
for i in range(len(classification_list)):
    if classification_counts[i] <= cutoff_value:
        classifications_to_replace.append(classification_list[i])

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1197
C7000      777
C1700      287
Name: count, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_dummies = pd.get_dummies(application_df,dtype='int64')

In [10]:
# Split our preprocessed data into our features and target arrays
y = application_df_dummies['IS_SUCCESSFUL'].values
X = application_df_dummies.drop(columns=['IS_SUCCESSFUL']).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  2080
hidden_nodes_layer2 = 512
hidden_nodes_layer3 = 64
hidden_nodes_layer4 = 8

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 2080)              95680     
                                                                 
 dense_1 (Dense)             (None, 512)               1065472   
                                                                 
 dense_2 (Dense)             (None, 64)                32832     
                                                                 
 dense_3 (Dense)             (None, 8)                 520       
                                                                 
 dense_4 (Dense)             (None, 1)                 9         
                                                                 
Total params: 1,194,513
Trainable params: 1,194,513
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [14]:
# Train the model
with tf.device('/GPU:0'):  #Note: I set up tensorflow with GPU. Change this to CPU:0 in order to obtain standard use. 
    fit_model = nn.fit(X_train_scaled,y_train,epochs=300)

Epoch 1/300
804/804 [==============================] - 6s 7ms/step - loss: 0.5728 - accuracy: 0.7185
Epoch 2/300
804/804 [==============================] - 5s 6ms/step - loss: 0.5573 - accuracy: 0.7221
Epoch 3/300
804/804 [==============================] - 5s 6ms/step - loss: 0.5551 - accuracy: 0.7217
Epoch 4/300
804/804 [==============================] - 5s 6ms/step - loss: 0.5519 - accuracy: 0.7299
Epoch 5/300
804/804 [==============================] - 5s 6ms/step - loss: 0.5502 - accuracy: 0.7268
Epoch 6/300
804/804 [==============================] - 5s 6ms/step - loss: 0.5487 - accuracy: 0.7289
Epoch 7/300
804/804 [==============================] - 5s 6ms/step - loss: 0.5475 - accuracy: 0.7321
Epoch 8/300
804/804 [==============================] - 5s 6ms/step - loss: 0.5470 - accuracy: 0.7327
Epoch 9/300
804/804 [==============================] - 5s 7ms/step - loss: 0.5458 - accuracy: 0.7333
Epoch 10/300
804/804 [==============================] - 5s 6ms/step - loss: 0.5453 - accura

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.7882 - accuracy: 0.7252 - 505ms/epoch - 2ms/step
Loss: 0.7882203459739685, Accuracy: 0.7252478003501892


In [16]:
# Export our model to HDF5 file
nn.save('Model_RC_Chlng21.h5')


# Module 21 report:

## Overview
#### This analysis looked at a dataset of venture capital applicants. The goal is to create an model than can take concrete data about the applicant such as the affiliation, classification, use case, organization, income, and desired capital and accurately determine the likelyhood of the applicant's success and a return on investment. 

## Results
### Data Processing
##### 	* The target is the "is successful" label of the dataset. The applicant needs to be successful for the venture to be worthwhile.
##### 	* The features are the remaining terms mentioned earlier: the affiliation, classification, use case, organization, income, and desired capital
##### 	* The remaining columns are then the EIN and name columns, which are simply descriptors of the organizations. 

### Compiling, training, and evaluation
##### 	* My model ended up being comprised of 4 hidden layers with 2080, 512, 64, and 8 nodes respectively. I used relu activation for all of the hidden layers and a sigmoid activation for the output layer. I tested switching to tanh activation functions within the output and hidden layers, but saw little to no change. What actually appeared the most "efficient" was having about 2x the number of terms as nodes per layer. However, I also wanted to test how many terms I could add without bogging down computation time dramatically, especially since I got GPU compute working locally, and tested it outside of Google Colab. Around 500 nodes were required before epoch times notably increased from 1ms/step. From here I increased the epochs and the nodes/layers to a reasonable training duration. 
##### 	* In the end, I was not able to achieve the target performance. I wanted to achieve the performance via increasing model complexity instead of adjusting parameters directly (to my own folly). 
##### 	* Increase Epochs
##### 	* Increase hidden layers
##### 	* Adjusted activation functions to tanh (and back)
##### 	* Adjusted cut-off values for the data bins. 
	
## Summary
#### Overall, I achieved an accuracy of 0.7433, and I am somewhat satisfied with that value. I wouldn't recommend this specific model however, as I believe the capital utilization may still be more effectively determined by human values. I believe that a neural network is still likely to be a good model choice for the dataset however, as it should be able to correlate the inputs to trends more effectively that a purely math-based fit model.
